In [1]:
import gurobipy as gp

In [31]:
qtde_itens = 63
qtde_mochilas = 3

vet_pesos = [145, 159, 148, 128, 80, 92, 141, 178, 93, 146, 64, 129, 83, 84, 127, 168, 75, 80, 153, 133, 110, 79, 105, 87, 87, 53, 158, 146, 51, 107, 116, 165, 167, 134, 169, 62, 57, 127, 54, 199, 115, 108, 138, 81, 75, 169, 148, 191, 82, 123, 189, 189, 146, 55, 140, 88, 154, 157, 106, 53, 192, 180, 177]
vet_valores = [326, 417, 333, 345, 202, 209, 363, 404, 232, 350, 172, 309, 213, 201, 311, 424, 174, 204, 397, 302, 253, 215, 270, 230, 221, 125, 430, 397, 132, 272, 281, 400, 375, 314, 447, 150, 128, 339, 139, 542, 290, 251, 317, 212, 168, 452, 399, 453, 223, 319, 505, 448, 365, 127, 318, 200, 400, 365, 262, 137, 441, 423, 469]
vet_capacidades = [1500, 2000, 1700]

itens = ["Item_{}".format(i+1) for i in range(qtde_itens)]
mochilas = ["Mochila_{}".format(i+1) for i in range(qtde_mochilas)]

capacidades = dict((mochilas[i], valor) for i, valor in enumerate(vet_capacidades))
pesos = dict((itens[i], peso) for i, peso in enumerate(vet_pesos))
valores = dict((itens[i], valor) for i, valor in enumerate(vet_valores))

In [38]:
m = gp.Model()

# Variaveis de decisão
x = m.addVars(itens, mochilas,  vtype=gp.GRB.BINARY)

# Função Objetivo do problema
m.setObjective(
    gp.quicksum(x[i, j] * valores[i] for i in itens for j in mochilas),
    sense=gp.GRB.MAXIMIZE
)

#restrições
c1 = m.addConstrs(
    gp.quicksum(x[i, j] for j in mochilas) <= 1 for i in itens
)

c2 = m.addConstrs(
    gp.quicksum(x[i, j] * pesos[i] for i in itens) <= capacidades[j] for j in mochilas
)

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 66 rows, 189 columns and 378 nonzeros
Model fingerprint: 0x653252e3
Variable types: 0 continuous, 189 integer (189 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 12745.000000
Presolve time: 0.00s
Presolved: 66 rows, 189 columns, 378 nonzeros
Variable types: 0 continuous, 189 integer (189 binary)

Root relaxation: objective 1.337307e+04, 75 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 13373.0741    0    5 12745.0000 13373.0741  4.93%     -    0s
H    0     0                    13313.000000 13373.0741  0.45%     -    0s
     0     0 13373.07

In [48]:
print("Valor Total:", m.objVal)
print()

for j in mochilas:
    print(j)
    for i in itens :
        if round(x[i, j].X) == 1 :
            print(i, end='  ')
    print("\n")     

Valor Total: 13364.0

Mochila_1
Item_5  Item_9  Item_13  Item_14  Item_18  Item_22  Item_24  Item_25  Item_27  Item_29  Item_30  Item_44  Item_49  Item_51  Item_59  Item_60  

Mochila_2
Item_12  Item_15  Item_16  Item_28  Item_32  Item_35  Item_38  Item_40  Item_46  Item_50  Item_53  Item_57  Item_63  

Mochila_3
Item_2  Item_4  Item_7  Item_10  Item_11  Item_19  Item_23  Item_31  Item_39  Item_41  Item_47  Item_48  Item_62  



In [51]:
print('Capacidade utilizada:')
for j in mochilas:
    print(j, capacidades[j] - c2[j].Slack )

Capacidade utilizada:
Mochila_1 1500.0
Mochila_2 1999.0
Mochila_3 1700.0
